In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import sys
sys.path.append("../../../libs_bell")
from lib_preprocessing import job_ids_to_result, flatten_results_jobs_list, arrange_results_list_tensored3, results_list_to_hist_list

In [3]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [4]:
from libs_qrem import MooneyEtalFilter
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [5]:
max_size = 65
max_length = 30
mooney001_mitigator_list = []
for n in range(2, max_size + 1):
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 1], mit_pattern=mit_pattern)
    mooney001_mitigator_list.append(MooneyEtalFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of mooney001_mitigator_list: ", len(mooney001_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
length of mooney001_mitigator_list:  64


In [6]:
for i in range(max_length):
    t1 = time.perf_counter()
    _ = mooney001_mitigator_list[i].apply(raw_hist_list[i + 1], threshold = 0.01)
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 7.708300000075496e-05 s )
2 th finished ( 3 qubits, 0.00010199399999955006 s )
3 th finished ( 4 qubits, 0.00016348099999952126 s )
4 th finished ( 5 qubits, 0.0002477340000002215 s )
5 th finished ( 6 qubits, 0.00029066199999938647 s )
6 th finished ( 7 qubits, 0.0005282439999998445 s )
7 th finished ( 8 qubits, 0.0008106300000001454 s )
8 th finished ( 9 qubits, 0.0013656379999993362 s )
9 th finished ( 10 qubits, 0.0018931660000003347 s )
10 th finished ( 11 qubits, 0.0022654630000005227 s )
11 th finished ( 12 qubits, 0.003359564000000148 s )
12 th finished ( 13 qubits, 0.004283639000000505 s )
13 th finished ( 14 qubits, 0.006300589000000301 s )
14 th finished ( 15 qubits, 0.008022056000001498 s )
15 th finished ( 16 qubits, 0.009287386000000453 s )
16 th finished ( 17 qubits, 0.01321792700000124 s )
17 th finished ( 18 qubits, 0.01537779300000075 s )
18 th finished ( 19 qubits, 0.018456416999999448 s )
19 th finished ( 20 qubits, 0.022559669000001392 s )

In [7]:
mooney001_mitigator_info = []
for i in range(max_length):
    t1 = time.perf_counter()
    mooney001_mitigator_info.append({# "exact_one_norm_of_reduced_inv_A": mooney001_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                                 "mitigated_hist": mooney001_mitigator_list[i].mitigated_hist(),
                                 "x_s": mooney001_mitigator_list[i].x_s(),
                                 # "x_hat": mooney001_mitigator_list[i].x_hat(),
                                 "x_tilde": mooney001_mitigator_list[i].x_tilde(),
                                 "sum_of_x": mooney001_mitigator_list[i].sum_of_x(),
                                 # "sum_of_x_hat": mooney001_mitigator_list[i].sum_of_x_hat(),
                                 "sum_of_x_tilde": mooney001_mitigator_list[i].sum_of_x_tilde(),
                                 "indices_to_keys_vector": mooney001_mitigator_list[i].indices_to_keys_vector(),
                                 "times": mooney001_mitigator_list[i].times(),
                                 "expval": mooney001_mitigator_list[i].expval(),
                                 # "mitigation_stddev": mooney001_mitigator_list[i].mitigation_stddev(norm_type = "exact"),
                                 })
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 2.058900000001529e-05 s )
2 th finished ( 3 qubits, 1.1950999999399414e-05 s )
3 th finished ( 4 qubits, 1.2181000000666131e-05 s )
4 th finished ( 5 qubits, 1.4274999999841498e-05 s )
5 th finished ( 6 qubits, 9.350000000907244e-06 s )
6 th finished ( 7 qubits, 1.6122999999979015e-05 s )
7 th finished ( 8 qubits, 1.3512999998965824e-05 s )
8 th finished ( 9 qubits, 1.242699999970398e-05 s )
9 th finished ( 10 qubits, 1.2837999999959493e-05 s )
10 th finished ( 11 qubits, 1.2513999999796965e-05 s )
11 th finished ( 12 qubits, 1.2726999999657096e-05 s )
12 th finished ( 13 qubits, 9.601000000358795e-06 s )
13 th finished ( 14 qubits, 1.1685999998789498e-05 s )
14 th finished ( 15 qubits, 1.358199999934584e-05 s )
15 th finished ( 16 qubits, 1.0916000000804615e-05 s )
16 th finished ( 17 qubits, 1.3616000000382655e-05 s )
17 th finished ( 18 qubits, 8.771000000962204e-06 s )
18 th finished ( 19 qubits, 1.3113999999703196e-05 s )
19 th finished ( 20 qubits, 1.826

In [8]:
with open("./pkls/mooney001_mitigator_info.pkl", "wb") as f:
    pickle.dump(mooney001_mitigator_info, f)